# Numerical SOS decomposition for $\beta_T$

## Preliminaries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ncpol2sdpa as ncp
import pandas as pd

# Define quantum operators
A_config = [2,2]
B_config = [2,2]

# Operators in problem
A = [Ai for Ai in ncp.generate_measurements(A_config, 'A')]
B = [Bj for Bj in ncp.generate_measurements(B_config, 'B')]

ops = ncp.flatten([A,B])        # Base monomials involved in problem
subs = {}

# 1. Unitarity: A^2 = 1
for op in ops:
    subs[op**2] = 1

# 2. Commutation: force a canonical ordering B*A → A*B
for Ai in ncp.flatten(A):
    for Bj in ncp.flatten(B):
        subs[Bj * Ai] = Ai * Bj

A0 = A[0][0]
A1 = A[1][0]
B0 = B[0][0]
B1 = B[1][0]

### Debugging

In [22]:
print(A0 * A1 == A1 * A0)  # This should print: False
print(B0 * B1 == B1 * B0)  # This should print: False
print(A0 * B0 == B0 * A0)  # This should print: False (after, we will guarantee [A_x, B_y] = 0 via subs)
print(A0 * B1 == B1 * A0)  # This should print: False (after, we will guarantee [A_x, B_y] = 0 via subs)

False
False
False
False


In [15]:
from sympy.physics.quantum.operator import HermitianOperator

for i, row in enumerate(A):
    for j, op in enumerate(row):
        print(f"A[{i}][{j}] is Hermitian? {isinstance(op, HermitianOperator)}")
for i, row in enumerate(B):
    for j, op in enumerate(row):
        print(f"B[{i}][{j}] is Hermitian? {isinstance(op, HermitianOperator)}")

A[0][0] is Hermitian? True
A[1][0] is Hermitian? True
B[0][0] is Hermitian? True
B[1][0] is Hermitian? True


In [16]:
subs

{A0**2: 1,
 A1**2: 1,
 B0**2: 1,
 B1**2: 1,
 B0*A0: A0*B0,
 B1*A0: A0*B1,
 B0*A1: A1*B0,
 B1*A1: A1*B1}

## Test: the CHSH expression

### $T_{1+A+B+AB}$

In [38]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)
    
    # AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()    # extra monomials

obj = (A0*B0 + A1*B0 + A0*B1 - A1*B1)/(2*np.sqrt(2))

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)

1.0000000003262057 0.999999999787619 optimal


In [39]:
print("The monomial basis is: ", sdp.monomial_sets[0])
print(" ")

Wmonchsh = sdp.y_mat[0]
dfchsh = pd.DataFrame(Wmonchsh)
np.savetxt("Wmonchsh.csv", Wmonchsh, delimiter=",")
print(dfchsh.to_string(float_format="{:.6f}".format))
print(" ")

sos_chsh = sdp.get_sos_decomposition()
print("SOS decomposition in the monomial basis: ", sos_chsh)

The monomial basis is:  [1, A0, A1, B0, B1, A0*B0, A0*B1, A1*B0, A1*B1]
 
          0         1         2         3         4         5         6         7         8
0  0.276783 -0.000000  0.000000 -0.000000 -0.000000  0.097857  0.097857  0.097857 -0.097857
1 -0.000000  0.111609 -0.000000  0.078919  0.078919  0.000000 -0.000000  0.000000  0.000000
2  0.000000 -0.000000  0.111609  0.078919 -0.078919  0.000000 -0.000000  0.000000  0.000000
3 -0.000000  0.078919  0.078919  0.111609 -0.000000  0.000000 -0.000000  0.000000  0.000000
4 -0.000000  0.078919 -0.078919 -0.000000  0.111609 -0.000000  0.000000 -0.000000 -0.000000
5  0.097857  0.000000  0.000000  0.000000 -0.000000  0.069195  0.034598  0.034598 -0.000000
6  0.097857 -0.000000 -0.000000 -0.000000  0.000000  0.034598  0.069195 -0.000000 -0.034598
7  0.097857  0.000000  0.000000  0.000000 -0.000000  0.034598 -0.000000  0.069195 -0.034598
8 -0.097857  0.000000  0.000000  0.000000 -0.000000 -0.000000 -0.034598 -0.034598  0.069195
 
SOS 

### $T_{1+A+B+AB+ABB'}$

In [20]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)
    
    # AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()    # extra monomials

obj = (A0*B0 + A1*B0 + A0*B1 - A1*B1)/(2*np.sqrt(2))

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)

1.0000000006196237 0.99999999973541 optimal


In [21]:
print("The monomial basis is: ", sdp.monomial_sets[0])
print(" ")

Wmonchsh = sdp.y_mat[0]
dfchsh = pd.DataFrame(Wmonchsh)
np.savetxt("Wmonchsh.csv", Wmonchsh, delimiter=",")
print(dfchsh.to_string(float_format="{:.6f}".format))
print(" ")

sos_chsh = sdp.get_sos_decomposition()
print("SOS decomposition in the monomial basis: ", sos_chsh)

The monomial basis is:  [1, A0, A1, B0, B1, A0*B0, A0*B1, A1*B0, A1*B1, A0*B0*B1, A0*B1*B0, A1*B0*B1, A1*B1*B0]
 
          0         1         2         3         4         5         6         7         8         9         10        11        12
0   0.149788 -0.000000 -0.000000 -0.000000  0.000000  0.052958  0.052958  0.052958 -0.052958 -0.000000 -0.000000 -0.000000 -0.000000
1  -0.000000  0.082726  0.000000  0.061273  0.061273  0.000000 -0.000000  0.000000  0.000000  0.021695  0.021695 -0.001963  0.001963
2  -0.000000  0.000000  0.082726  0.061273 -0.061273  0.000000 -0.000000 -0.000000  0.000000  0.001963 -0.001963 -0.021695 -0.021695
3  -0.000000  0.061273  0.061273  0.175107 -0.000000  0.000000 -0.000000 -0.000000  0.000000  0.062546  0.000000 -0.062546  0.000000
4   0.000000  0.061273 -0.061273 -0.000000  0.175107  0.000000  0.000000  0.000000  0.000000 -0.000000  0.062546  0.000000  0.062546
5   0.052958  0.000000  0.000000  0.000000  0.000000  0.041374  0.018723  0.018723  0.00

Two lessons: 
- Do not use ``get_sos_decomposition()`` since it does not seem to take into account the ``subs``.
- The dual solution works, but only assuming $[A_0, A_1] = 0, \quad [B_0, B_1] = 0$.

## Testing Victor's Bell expressions at different levels of the hierarchy

In [ ]:
def test_point_quantum_bound(r0, r1, level=0, verbose=False):
    """
    Tests whether a point (parametrized by theta, r0 = p1, r1 = 2*p4) is quantum-realizable
    by solving the associated SDP relaxation at level 0 with different sets of monomials.

    Parameters:
    -----------
    theta : float
        The measurement angle θ in radians.
    r0 : float
        Parameter such that p1 = r0.
    r1 : float
        Parameter such that p4 = 0.5 * r1.
    level : str or int
        Level of the relaxation: 0, 'AB', 'ABB', or 'AAB'.
    solver : str
        SDP solver to use (e.g., "mosek", "cvxopt", "sdpa").
    verbose : bool
        Whether to print verbose SDP solver output.

    Returns:
    --------
    primal : float
        The negative primal bound.
    dual : float
        The negative dual bound.
    status : str
        The solver status string.
    """

    obj = r0*((A[0][0] + A[1][0])/np.sqrt(2) - B[0][0]) + r1*((A[0][0] - A[1][0])/np.sqrt(2) - B[1][0]) + (A[0][0]*B[0][0] + A[1][0]*B[0][0] + A[0][0]*B[1][0] - A[1][0]*B[1][0])/(2*np.sqrt(2))

    def get_extra_monomials(level):
        """
        Generate extra monomials for a given level string such as:
        'AA', 'BB', 'AB', 'AAB', 'ABB', 'AA+BB+AB+AAB+ABB', etc.
        """
        if level in [0, '0', None]:
            return []

        level_parts = level.split('+') if isinstance(level, str) else []
        monos = []
        Aflat = ncp.flatten(A)
        Bflat = ncp.flatten(B)

        if 'AA' in level_parts:
            for a1 in Aflat:
                for a2 in Aflat:
                    monos.append(a1 * a2)

        if 'BB' in level_parts:
            for b1 in Bflat:
                for b2 in Bflat:
                    monos.append(b1 * b2)

        if 'AB' in level_parts:
            for a in Aflat:
                for b in Bflat:
                    monos.append(a * b)

        if 'AAB' in level_parts:
            for a1 in Aflat:
                for a2 in Aflat:
                    for b in Bflat:
                        monos.append(a1 * a2 * b)

        if 'ABB' in level_parts:
            for a in Aflat:
                for b1 in Bflat:
                    for b2 in Bflat:
                        monos.append(a * b1 * b2)

        return monos[:]

    extra_monos = get_extra_monomials(level)

    # Create and solve SDP relaxation at fixed level=0
    sdp = ncp.SdpRelaxation(ops, verbose=int(verbose), normalized=True, parallel=0)
    sdp.get_relaxation(
        level=0,
        equalities=op_eqs[:],
        inequalities=op_ineqs[:],
        momentequalities=moment_eqs[:],
        momentinequalities=moment_ineqs[:],
        objective=-obj,
        substitutions=subs,
        extramonomials=extra_monos
    )
    sdp.solve(solver="mosek")

    return -sdp.primal, -sdp.dual, sdp.status

In [ ]:
r0 = 1 - 1/np.sqrt(2)
r1 = 0

for lvl in [0, 'AA', 'BB', 'AB', 'BB+AB', 'ABB', 'AAB', 'AB+ABB', 'BB+AB+ABB']:
    p, d, s = test_point_quantum_bound(r0, r1, level=lvl)
    print(f"Level {lvl}: primal = {p:.10f}, dual = {d:.10f}, status = {s}")

## Extracting the certificate at $T_{1+A+B+AB+ABB}$

We are able to reach $\beta_T$ at this level of the hierarchy.

In [2]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)
    
    # AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()   # extra monomials


r0 = 1 - 1/np.sqrt(2) # p1 = r0
r1 = 0 # p4 = 1/2 r1

obj = r0*((A[0][0] + A[1][0])/np.sqrt(2) - B[0][0]) + r1*((A[0][0] - A[1][0])/np.sqrt(2) - B[1][0]) + (A[0][0]*B[0][0] + A[1][0]*B[0][0] + A[0][0]*B[1][0] - A[1][0]*B[1][0])/(2*np.sqrt(2))

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")

if sdp.primal is not None and sdp.dual is not None:
    print(-sdp.primal, -sdp.dual, sdp.status)
else:
    print("Solver did not return a solution. Status:", sdp.status)

1.00000000835675 0.9999999939876847 optimal


In [43]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    #monos = [A0**2, A0, A1, B0, A0*B0, A1*B0, A0*A1*B0, A1*A0*B0, B1, A0*B1, A1*B1, A0*A1*B1, A1*A0*B1]
    monos = [A0**2, B0, B1, A0, B0*A0, B1*A0, B0*B1*A0, B1*B0*A0, A1, B0*A1, B1*A1, B0*B1*A1, B1*B0*A1]
    #Aflat = ncp.flatten(A)
    #Bflat = ncp.flatten(B)

    #monos += Aflat + Bflat  # Include all base monomials
    '''
    # AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    '''
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = [B0 + 1, B1 - 1]                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()   # extra monomials


r0 = 1 - 1/np.sqrt(2) # p1 = r0
r1 = 0 # p4 = 1/2 r1

obj = r0*((A[0][0] + A[1][0])/np.sqrt(2) - B[0][0]) + r1*((A[0][0] - A[1][0])/np.sqrt(2) - B[1][0]) + (A[0][0]*B[0][0] + A[1][0]*B[0][0] + A[0][0]*B[1][0] - A[1][0]*B[1][0])/(2*np.sqrt(2))

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=-1,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")

if sdp.primal is not None and sdp.dual is not None:
    print(-sdp.primal, -sdp.dual, sdp.status)
else:
    print("Solver did not return a solution. Status:", sdp.status)

1.0000000118213126 0.9999999956859023 optimal


In [44]:
sdp.monomial_sets

[[A0**2,
  B0,
  B1,
  A0,
  B0*A0,
  B1*A0,
  B0*B1*A0,
  B1*B0*A0,
  A1,
  B0*A1,
  B1*A1,
  B0*B1*A1,
  B1*B0*A1]]

In [48]:
Wmon = sdp.y_mat[0]
df = pd.DataFrame(Wmon)
print(df.to_string(float_format="{:.6f}".format))
np.savetxt("Wmon.csv", Wmon, delimiter=",")

          0         1         2         3         4         5         6         7         8         9         10        11        12
0   1.718885  0.842801 -0.828876 -0.060187 -0.107450 -0.056955 -0.008907 -0.009266 -0.033573 -0.042170  0.041519 -0.002570 -0.002974
1   0.842801  1.018221  0.024868 -0.121196 -0.097418  0.012696 -0.068107  0.000000 -0.014670 -0.036124  0.003773  0.118767  0.000000
2  -0.828876  0.024868  1.009545 -0.051715  0.005477  0.054051 -0.000000  0.051870  0.016491  0.001771 -0.033856  0.000000 -0.119936
3  -0.060187 -0.121196 -0.051715  1.741808  0.890401 -0.876854 -0.006532 -0.013039 -0.013206 -0.011310  0.007219 -0.000734  0.002643
4  -0.107450 -0.097418  0.005477  0.890401  1.744444  0.013839 -0.846909 -0.008472 -0.007502 -0.017762 -0.003701 -0.005638  0.001584
5  -0.056955  0.012696  0.054051 -0.876854  0.013839  1.737956  0.001443  0.848784  0.007115 -0.004829 -0.016349  0.000723  0.013432
6  -0.008907 -0.068107 -0.000000 -0.006532 -0.846909  0.001443  0.920

In [46]:
Gammamon = sdp.x_mat[0]
df = pd.DataFrame(Gammamon)
print(df.to_string(float_format="{:.6f}".format))
#np.savetxt("Wmon.csv", Wmon, delimiter=",")

          0         1         2         3         4         5         6         7         8         9         10        11        12
0   1.000000 -1.000000  1.000000  1.000000 -1.000000  1.000000 -1.000000 -1.000000 -1.000000  1.000000 -1.000000  1.000000  1.000000
1  -1.000000  1.000000 -1.000000 -1.000000  1.000000 -1.000000  1.000000  1.000000  1.000000 -1.000000  1.000000 -1.000000 -1.000000
2   1.000000 -1.000000  1.000000  1.000000 -1.000000  1.000000 -1.000000 -1.000000 -1.000000  1.000000 -1.000000  1.000000  1.000000
3   1.000000 -1.000000  1.000000  1.000000 -1.000000  1.000000 -1.000000 -1.000000 -1.000000  1.000000 -1.000000  1.000000  1.000000
4  -1.000000  1.000000 -1.000000 -1.000000  1.000000 -1.000000  1.000000  1.000000  1.000000 -1.000000  1.000000 -1.000000 -1.000000
5   1.000000 -1.000000  1.000000  1.000000 -1.000000  1.000000 -1.000000 -1.000000 -1.000000  1.000000 -1.000000  1.000000  1.000000
6  -1.000000  1.000000 -1.000000 -1.000000  1.000000 -1.000000  1.000

In [ ]:
sos_mon = sdp.get_sos_decomposition()
print("SOS decomposition in the monomial basis: ", sos_mon)

**Remark:** This expression for the SOS decomposition does not seem to take into account the properties of the operators we defined. It's better to not use it!

In [ ]:
sdp.monomial_sets[0]